In [1]:
import torch

from torch.utils.data import DataLoader, Dataset


import pandas as pd
import numpy as np
from gmf import GMFEngine, GMF
from mlp import MLPEngine
from neumf import NeuMFEngine
from data import SampleGenerator
from metrics import MetronAtK

import copy

In [2]:


def evaluate_hit_ndcg(model, evaluate_data):    
    model.eval()
    with torch.no_grad():
        test_users, test_items = evaluate_data[0], evaluate_data[1]
        negative_users, negative_items = evaluate_data[2], evaluate_data[3]        
        test_scores = model(test_users, test_items)
        negative_scores = model(negative_users, negative_items)
        
        metron = MetronAtK(top_k=10)
        
        metron.subjects = [test_users.data.view(-1).tolist(),
                             test_items.data.view(-1).tolist(),
                             test_scores.data.view(-1).tolist(),
                             negative_users.data.view(-1).tolist(),
                             negative_items.data.view(-1).tolist(),
                             negative_scores.data.view(-1).tolist()]
    hit_ratio, ndcg = metron.cal_hit_ratio(), metron.cal_ndcg()
        
    
    return hit_ratio, ndcg


def eval_ce_loss(model, data_loader):
    total_loss = 0.0
    n_batches = 0
    model.eval()
    with torch.no_grad():
        for batch in data_loader:
            users, items, ratings = batch[0], batch[1], batch[2]
            ratings = ratings.float()
            ratings_pred = model(users, items)
            crit = torch.nn.BCELoss()
            loss = crit(ratings_pred.view(-1), ratings)

            total_loss += loss.item()
            n_batches += 1
    
    return total_loss / n_batches

In [3]:
ml1m_dir = 'data/ml-100k/u.data'
ml1m_rating = pd.read_csv(ml1m_dir, sep='\t', header=None, names=['uid', 'mid', 'rating', 'timestamp'],  engine='python')
# Reindex
user_id = ml1m_rating[['uid']].drop_duplicates().reindex()
user_id['userId'] = np.arange(len(user_id))
ml1m_rating = pd.merge(ml1m_rating, user_id, on=['uid'], how='left')
item_id = ml1m_rating[['mid']].drop_duplicates()
item_id['itemId'] = np.arange(len(item_id))
ml1m_rating = pd.merge(ml1m_rating, item_id, on=['mid'], how='left')
ml1m_rating = ml1m_rating[['userId', 'itemId', 'rating', 'timestamp']]
print('Range of userId is [{}, {}]'.format(ml1m_rating.userId.min(), ml1m_rating.userId.max()))
print('Range of itemId is [{}, {}]'.format(ml1m_rating.itemId.min(), ml1m_rating.itemId.max()))

Range of userId is [0, 942]
Range of itemId is [0, 1681]


In [4]:
num_users = len(ml1m_rating.userId.unique())
num_items = len(ml1m_rating.itemId.unique())

gmf_config = {
              'num_users': num_users,
              'num_items': num_items,
              'latent_dim': 10,
              'num_negative': 5,
              'l2_regularization': 0, # 0.01
              'use_cuda': False,
              'device_id': 0,
              'model_dir':'checkpoints/{}_Epoch{}_HR{:.4f}_NDCG{:.4f}.model'}





## federated training

In [5]:
class DatasetSplit(Dataset):
    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.idxs = list(idxs)

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        return self.dataset[self.idxs[item]]


class LocalUpdate:
    def __init__(self, model_global, dataset, user_id):
        idxs = torch.where(dataset.user_tensor == user_id)[0]
        self.local_dataset = DatasetSplit(dataset, idxs)
        self.num_local = len(self.local_dataset)        
        self.local_model = copy.deepcopy(model_global)
        self.user_id = user_id
        
        
    def train(self, lr, epoches_local, local_batch_size):
        local_data_loader = DataLoader(self.local_dataset, 
                                       batch_size=local_batch_size, 
                                       shuffle=True)
        
        self.local_model.train()
        opt = torch.optim.Adam(self.local_model.parameters(), lr=lr)
        for ep_local in range(epoches_local):
            t_loss = 0.0
            n_batch = 0
            for users, items, ratings in local_data_loader:
                opt.zero_grad()
                ratings = ratings.float()
                ratings_pred = self.local_model(users, items).view(-1)
                
                crit = torch.nn.BCELoss()
                loss = crit(ratings_pred, ratings)
                loss.backward()
                opt.step()
                
                t_loss += loss.item()
                n_batch += 1
            
            if (ep_local + 1) % 100  == 0:
                print("user", self.user_id, "ep_local", ep_local, t_loss/n_batch)
        
        return t_loss/n_batch


In [6]:
sample_generator = SampleGenerator(ratings=ml1m_rating)

In [32]:
model_global = GMF(gmf_config)

In [33]:
k_u, k_i = 'embedding_user.weight', 'embedding_item.weight'

In [34]:
# def train_fedavg(model_global, sample_generator):
frac = 0.1
epoches_local = 3
local_lr = 1e-2
act_samp = False
n_clusters = 5

for epoch in range(500):
    train_loader = sample_generator.instance_a_train_loader(num_negatives=gmf_config['num_negative'], 
                                                            batch_size=1024)
    train_dataset = train_loader.dataset
    evaluate_data = sample_generator.evaluate_data

    if epoch > 5:
        act_samp = True
    # sample users    
    if act_samp:
        idxs_users = []
        n_clusters = 5
        kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(model_global.state_dict()[k_u].numpy())
        for c in range(n_clusters):
            c_idxs = np.where(kmeans.labels_ == c)[0]
            sel = np.random.choice(c_idxs, max(int(frac * len(c_idxs)), 1))            
            idxs_users.append(sel)

        idxs_users = np.concatenate(idxs_users)
        m = len(idxs_users)
        
    else:
        m = max(int(frac * num_users), 1)
        idxs_users = np.random.choice(range(num_users), m, replace=False)

    w_locals, n_locals = [], []
    total_loss = 0.0
    for idx in idxs_users:        
        # local update
        local_update = LocalUpdate(model_global, train_dataset, user_id=idx)
        local_batch_size = int(len(local_update.local_dataset)*0.1)
        total_loss += local_update.train(local_lr, epoches_local, local_batch_size=local_batch_size)

        w_locals.append(copy.deepcopy(local_update.local_model.state_dict()))
        n_locals.append(local_update.num_local)
    
    # aggregated model
    w_avg = {}
    
    # keys of user and item embeddings in the model state dict

    w_global = model_global.state_dict()
    
    # aggregated user embeddings
    # line 13 - 16 of Algorithm 3
    # user embedding: the k-th chosen user only updates its own user embedding
    w_avg[k_u] = copy.deepcopy(w_global[k_u])
    for k in range(m):
        w_avg[k_u][idxs_users[k]] = w_locals[k][k_u][idxs_users[k]]
            
    # contribution of each user to all items, measured by L1 distance between the 
    # local item embeddings to the global item embeddings
    # line 6 - 9 of Algorithm 3
    contrib = torch.zeros(m, num_items)
    for k in range(m):
        contrib[k] = (w_global[k_i] - w_locals[k][k_i]).abs().sum(1)

    eps = 1e-10
    contrib = contrib / (contrib.sum(0) + eps)
    
    # aggregate item embeddings: line 10 - 12 of Algorithm 3
    w_avg[k_i] = copy.deepcopy(w_global[k_i])
    for i in range(num_items):
        if contrib[:, i].sum() == 0:
            # item i is not updated by any chosen user, keep the original embedding
            continue
        
        w_avg[k_i][i] *= 0
        for k in range(m):
            # aggregated embedding of item i: weighted by the contribution of each user
            w_avg[k_i][i] += w_locals[k][k_i][i] * contrib[k][i]
    
    # aggregate other model parameters
    # line 2 - 3 of algorithm 3
    n_locals = np.array(n_locals, dtype=np.float64)
    n_locals /= n_locals.sum()
    for k in w_global.keys():
        if k != k_u and k != k_i:
            w_avg[k] = 0.0
            for i in range(len(w_locals)):
                w_avg[k] += w_locals[i][k] * n_locals[i]
    

    model_global.load_state_dict(w_avg)
    hit_ratio, ndcg = evaluate_hit_ndcg(model_global, evaluate_data)

    total_loss_global = eval_ce_loss(model_global, train_loader)
    print(epoch, "local_loss", total_loss/m, "total_loss_global", total_loss_global, 
          "hit_ratio", hit_ratio, "ndcg", ndcg)






/Users/xianl/code/neural-collaborative-filtering/src/metrics.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(lambda x: math.log(2) / math.log(1 + x)) # the rank starts from 1


0 local_loss 0.5638173412596464 total_loss_global 0.5907297875097409 hit_ratio 0.20890774125132555 ndcg 0.09581606241279174
1 local_loss 0.5082355122905173 total_loss_global 0.5238193160182638 hit_ratio 0.20890774125132555 ndcg 0.09618022333213372
2 local_loss 0.46409606663816416 total_loss_global 0.483521461230342 hit_ratio 0.21208907741251326 ndcg 0.0946602157135031
3 local_loss 0.44241495090942057 total_loss_global 0.46256528176662226 hit_ratio 0.21739130434782608 ndcg 0.09749767557396498
4 local_loss 0.43361160115156927 total_loss_global 0.45534941297595144 hit_ratio 0.2067868504772004 ndcg 0.09565814257409042
5 local_loss 0.4264436323649647 total_loss_global 0.454107537121863 hit_ratio 0.2067868504772004 ndcg 0.0924855918115337


/Users/xianl/code/neural-collaborative-filtering/src/metrics.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(lambda x: math.log(2) / math.log(1 + x)) # the rank starts from 1


6 local_loss 0.43313683787340757 total_loss_global 0.4533418654985149 hit_ratio 0.20996818663838812 ndcg 0.09748676476078552


/Users/xianl/code/neural-collaborative-filtering/src/metrics.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(lambda x: math.log(2) / math.log(1 + x)) # the rank starts from 1


...

/Users/xianl/code/neural-collaborative-filtering/src/metrics.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(lambda x: math.log(2) / math.log(1 + x)) # the rank starts from 1


106 local_loss 0.3073060073219828 total_loss_global 0.35499832865088 hit_ratio 0.6086956521739131 ndcg 0.3403435277788526


/Users/xianl/code/neural-collaborative-filtering/src/metrics.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(lambda x: math.log(2) / math.log(1 + x)) # the rank starts from 1


107 local_loss 0.3007903630006785 total_loss_global 0.3540189500109464 hit_ratio 0.6097560975609756 ndcg 0.3414734145311794


KeyboardInterrupt: 

In [35]:
dim_emb = w_global[k_u].shape[1]

avg_cluster_update = torch.zeros(n_clusters, dim_emb)
num_cluster_dels = torch.zeros(n_clusters)

cluster_labels = kmeans.labels_
for k in range(len(idxs_users)):
    c = cluster_labels[idxs_users[k]]
    avg_cluster_update[c] += w_locals[k][k_u][idxs_users[k]] - w_global[k_u][idxs_users[k]]
    num_cluster_dels[c] += 1
            